![DLI Header](../images/DLI_Header.png)

# Parallelize with C++ Parallel Algorithms

In this notebook you will perform your first exercise: refactoring the `apply_stencil` and `initial_condition` functions to use C++ parallel algorithms and thus enabling GPU acceleration.

## Learning Objectives

By the time you complete this notebook you should:

- Be able to replace nested for-loops with C++ parallel algorithms and views.
- Be able to initialize data directly on the GPU for reduced communication overhead.
- Compile the application for GPU acceleration.
- Observe drastically increased bandwidth as a result of GPU acceleration.

---

## Exercise Overview

The goal of this exercise is to parallelize the `apply_stencil` and `initial_condition` implementations using the C++ parallel algorithms.

You will be working with [exercise1.cpp](./exercise1.cpp), which is near identical to the source code you viewed in the last notebook, but which contains some scaffolding to assist your work.

The only functions that need to be modified for this exercise are the `apply_stencil` and `initial_condition` functions. You will also need to add some additional library includes. You can page search (CTRL+F) the source code for "TODO" (there are 3 "TODOs") to see where in the file you need to work.

### Apply Stencil

In the serial implementation, raw loops are used:

```c++
double apply_stencil(double* u_new, double* u_old, grid g, parameters p) {
  double energy = 0.;
  for (long x = g.x_begin; x < g.x_end; ++x) {
    for (long y = g.y_begin; y < g.y_end; ++y) {
      energy += stencil(u_new, u_old, x, y, p);
    }
  }
  return energy;
}
```

Parallelize it using the C++ parallel algorithms by:
* Using `std::views::cartesian_product` to create a range ([documentation](https://en.cppreference.com/w/cpp/ranges/cartesian_product_view)).
* Using the `std::transform_reduce` algorithm (overload 3 from the [documentation](https://en.cppreference.com/w/cpp/algorithm/transform_reduce)).

While `std::views::cartesian_product` is part of C++23, most C++ tool chains do not provide an implementation of it yet.
For this tutorial, we provide a single header implementation of `cartesian_product` that is back ported to C++20.
The exercise templates include it for you via: 

```c++
#include <cartesian_product.hpp> // Brings C++23 std::views::cartesian_product to C++20
```

Once C++23 support includes it, you'll automatically get it via the `<ranges>` include.

### Initial Condition

To keep all memory on the device when offloading the C++ parallel algorithms, also parallelize the `initial_condition` function from its raw loop version using the `std::fill_n` ([documentation](https://en.cppreference.com/w/cpp/algorithm/fill_n)) parallel algorithm:

```c++
void initial_condition(double* u_new, double* u_old, long n) {
  // TODO: implement using parallel algorithms
  for (long i = 0; i < n; ++i) {
    u_old[i] = 0.;
    u_new[i] = 0.;
  }
}
```

---

## Visualize Function

As in the previous notebook, we will utilize the `visualize` function to observe the output of running the `heat` application. Execute the cell below to define it for use in this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#plt.style.use('dark_background') # Uncomment for dark background

def visualize(name = 'output'):
    f = open(name, 'rb')
    grid = np.fromfile(f, dtype=np.uint64, count=2, offset=0)

    nx = grid[0]
    ny = grid[1]

    times = np.fromfile(f, dtype=np.float64, count=1, offset=0)
    time = times[0]

    values = np.fromfile(f, dtype=np.float64, offset=0)
    assert len(values) == nx * ny, f'{len(values)} != {nx * ny}'
    values = values.reshape((nx, ny))

    print(f'Plotting grid {nx}x{ny}, t = {time}')

    plt.title(f'Temperature at t = {time:.3f} [s]')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.pcolormesh(values, cmap=plt.cm.jet, vmin=0, vmax=values.max())
    plt.colorbar()
    plt.savefig('output.png', transparent=True, bbox_inches='tight', dpi=300)

---

## Compile and Run Commands

While [exercise1.cpp](./exercise1.cpp) compiles and runs as provided, it produces incorrect results due to the incomplete `apply_stencil` and `intial_condition` implementations.

You can run the following 2 cells presently if you wish, but their intended use is for you to be able to compile and run [exercise1.cpp](./exercise1.cpp) after you have done your work.

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=g++ mpicxx -std=c++20 -Ofast -march=native -o heat exercise1.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=clang++ mpicxx -std=c++20 -Ofast -march=native -o heat exercise1.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

---

## Solution

To check your work, of if you get stuck, please view the solution for this exercise in [solutions/exercise1.cpp](solutions/exercise1.cpp).

### Compiling for the CPU

The following cells compile and run the solution for this exercise using different compilers, targeting CPU cores. The `nvc++` complication targets multiple CPU cores for the source code's parallel algorithms.

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=g++ mpicxx -std=c++20 -Ofast -march=native -DNDEBUG -o heat solutions/exercise1.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=clang++ mpicxx -std=c++20 -Ofast -march=native -DNDEBUG -o heat solutions/exercise1.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=nvc++ mpicxx -std=c++20 -stdpar=multicore -O4 -fast -march=native -Mllvm-fast -DNDEBUG -o heat solutions/exercise1.cpp
!mpirun -np 2 ./heat 256 256 16000
visualize()

### Compiling for the GPU

Below we compile for GPU acceleration.

When using `-stdpar=gpu`, NVC++ enables `-gpu=managed` by default to emulate Unified Memory on non-coherent platforms.

When combining this with MPI, we can tell UCX-based MPI implementations that Managed Memory used with MPI "mostly" resides on the device via the `UCX_RNDV_FRAG_MEM_TYPE=cuda` environment variable, which will force UCX to use device-pinned communication buffers instead of staging data communication via the host CPU.

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=nvc++ mpicxx -std=c++20 -stdpar=gpu -O4 -fast -march=native -Mllvm-fast -DNDEBUG -o heat solutions/exercise1.cpp
!UCX_RNDV_FRAG_MEM_TYPE=cuda mpirun -np 2 ./heat 256 256 16000
visualize()

The problem size used here is too small to benefit from GPU acceleration, and thus while the program ran correctly and in a similar amount of time to the CPU implementations, the bandwidth calculations were not particularly impressive.

Try running the program again with a larger problem size, for example `2048 2048 16000` and you will observe much more impressive results. Please note that changing the dimensions of the inputs will of course result in a different output and thus a different visualization.

If you don't mind waiting around a bit, you can try passing the same arguments to one of the CPU-only implementations above, for comparison.

---

## Next

Please proceed to [the next notebook](../04-Overlap/Overlap.ipynb).

![DLI Header](../images/DLI_Header.png)